In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns; sns.set(); sns.set_style('dark')

import json
import os
import datetime

from sklearn.model_selection import train_test_split
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
import cairosvg
from PIL import Image

2025-01-17 20:46:44.587402: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737146804.608207    4701 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737146804.614474    4701 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-17 20:46:44.637868: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
with open("../configs/config.json", "r") as file:
    config = json.load(file)

# Виведення конфігурації
print(config)

{'data': {'raw_path': '../data/raw/svg', 'png_path': '../data/precessed/png_images', 'precessed_path': '../data/precessed/tfrec', 'class_names': ['airplane', 'alarm clock', 'angel', 'ant', 'apple', 'arm', 'armchair', 'ashtray', 'axe', 'backpack', 'banana', 'barn', 'baseball bat', 'basket', 'bathtub', 'bear (animal)', 'bed', 'bee', 'beer-mug', 'bell', 'bench', 'bicycle', 'binoculars', 'blimp', 'book', 'bookshelf', 'boomerang', 'bottle opener', 'bowl', 'brain', 'bread', 'bridge', 'bulldozer', 'bus', 'bush', 'butterfly', 'cabinet', 'cactus', 'cake', 'calculator', 'camel', 'camera', 'candle', 'cannon', 'canoe', 'car (sedan)', 'carrot', 'castle', 'cat', 'cell phone', 'chair', 'chandelier', 'church', 'cigarette', 'cloud', 'comb', 'computer monitor', 'computer-mouse', 'couch', 'cow', 'crab', 'crane (machine)', 'crocodile', 'crown', 'cup', 'diamond', 'dog', 'dolphin', 'donut', 'door', 'door handle', 'dragon', 'duck', 'ear', 'elephant', 'envelope', 'eye', 'eyeglasses', 'face', 'fan', 'feather',

In [3]:
# Constants
IMAGE_SIZE = (config['model']['input_size'], config['model']['input_size'])
BATCH_SIZE = config['model']['batch_size']
NUM_CLASSES = config['model']['output_size']
PNG_DIR = config['data']['png_path']
TFREC_DIR = config['data']['precessed_path']
class_names = sorted(os.listdir(PNG_DIR))

In [4]:
import os
import nvidia.dali as dali
from nvidia.dali import pipeline_def
import nvidia.dali.fn as fn
import nvidia.dali.types as types
import nvidia.dali.plugin.tf as dali_tf
import tensorflow as tf

# Отримуємо список папок (класів) та зображень
class_names = sorted(os.listdir(PNG_DIR))
image_paths = []
labels = []

for class_id, class_name in enumerate(class_names):
    class_dir = os.path.join(PNG_DIR, class_name)
    for image_name in os.listdir(class_dir):
        image_paths.append(os.path.join(class_dir, image_name))
        labels.append(class_id)

# Параметри
num_threads = 4
device_id = 0  # ID GPU

# Пайплайн DALI для завантаження та обробки зображень
import os
import nvidia.dali as dali
from nvidia.dali import pipeline_def
import nvidia.dali.fn as fn
import nvidia.dali.types as types
import nvidia.dali.plugin.tf as dali_tf
import tensorflow as tf


# Отримуємо список папок (класів) та зображень
class_names = sorted(os.listdir(PNG_DIR))
image_paths = []
labels = []

for class_id, class_name in enumerate(class_names):
    class_dir = os.path.join(PNG_DIR, class_name)
    for image_name in os.listdir(class_dir):
        image_paths.append(os.path.join(class_dir, image_name))
        labels.append(class_id)

# Параметри
BATCH_SIZE = 32
IMAGE_SIZE = (224, 224)
num_threads = 4
device_id = 0  # ID GPU

# Пайплайн DALI для завантаження та обробки зображень
@pipeline_def
def create_pipeline(image_paths, labels):
    images, labels = fn.readers.file(
        files=image_paths,
        labels=labels,
        random_shuffle=True,
        name="Reader"
    )
    images = fn.decoders.image(
        images,
        device='mixed',  # Використовуємо GPU для декодування
        output_type=types.RGB
    )
    images = fn.resize(
        images,
        resize_x=IMAGE_SIZE[0],
        resize_y=IMAGE_SIZE[1],
        interp_type=types.INTERP_LINEAR
    )
    images = fn.crop_mirror_normalize(
        images,
        dtype=types.FLOAT,
        output_layout="HWC",
        crop=(IMAGE_SIZE[0], IMAGE_SIZE[1]),
        mean=[0.485 * 255, 0.456 * 255, 0.406 * 255],  # Нормалізація для ImageNet
        std=[0.229 * 255, 0.224 * 255, 0.225 * 255]
    )
    labels = labels.gpu()  # Переміщуємо мітки на GPU
    return images, labels

# Створюємо пайплайн
pipeline = create_pipeline(
    image_paths=image_paths,  # Передаємо шляхи до зображень
    labels=labels,           # Передаємо мітки
    batch_size=BATCH_SIZE,
    num_threads=num_threads,
    device_id=device_id
)

# Підключаємо DALI до TensorFlow
dali_dataset = dali_tf.DALIDataset(
    pipeline=pipeline,
    output_dtypes=(tf.float32, tf.int32),
    output_shapes=((IMAGE_SIZE[0], IMAGE_SIZE[1], 3), ())
)

I0000 00:00:1737146807.613887    4701 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5592 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060 Ti, pci bus id: 0000:05:00.0, compute capability: 8.6


In [5]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model

# Функція для створення моделі
def create_model():
    # Завантаження базової моделі
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(*IMAGE_SIZE, 3))
    
    # Заморожуємо базову модель
    base_model.trainable = False

    # Додаємо власні шари
    inputs = tf.keras.Input(shape=(*IMAGE_SIZE, 3))
    x = base_model(inputs, training=False)
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(NUM_CLASSES, activation='softmax')(x)

    model = Model(inputs, x)
    return model

# Створюємо модель
model = create_model()
model.summary()

# Компіляція моделі
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       655,872 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 250)            │       128,250 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,042,106 (11.60 MB)

 Trainable params: 784,122 (2.99 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [7]:
import time
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard

# Колбеки
callbacks = [
    EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6),
]

# Етап 1: Навчання верхніх шарів
print("Етап 1: Навчання верхніх шарів...")
start_time = time.time()

history_stage_2 = model.fit(
    dali_dataset,
    epochs=10,
    steps_per_epoch=len(image_paths) // BATCH_SIZE,
    callbacks=callbacks
)

print(f"Час навчання етапу 1: {time.time() - start_time:.2f} секунд")

Етап 1: Навчання верхніх шарів...
Epoch 1/10


: 

In [ ]:

# Розморожуємо верхні шари базової моделі
print("Розморожуємо верхні шари...")
base_model.trainable = True
for layer in base_model.layers[:-30]:  # Заморожуємо всі шари, окрім останніх 30
    layer.trainable = False

# Перекомпіляція моделі з низьким learning rate
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:

# Етап 2: Fine-tuning базової моделі
print("Етап 2: Fine-tuning базової моделі...")
start_time = time.time()

history_stage_2 = model.fit(
    dali_dataset,
    epochs=20,
    steps_per_epoch=len(image_paths) // BATCH_SIZE,
    callbacks=callbacks
)

print(f"Час навчання етапу 2: {time.time() - start_time:.2f} секунд")


In [ ]:

# Збереження моделі
model.save('./mobilenetv2_sketch_classifier.h5')

# Моніторинг процесу через TensorBoard
print("Запустіть TensorBoard для моніторингу навчання: tensorboard --logdir=./logs")
